# MCP Resources & Third-Party Server Integration

This notebook explores **MCP Resources** (context injection) and integration with external MCP servers. Three deployment models:
1. **Local-only**: No external dependencies (e.g., memory server)
2. **Local + API**: MCP server calls web services (e.g., Brave Search, Polygon.io)
3. **Remote/Hosted**: Fully managed MCP servers (rare in practice)

**Key Pattern:** MCP separates tool interface from implementation location, enabling hybrid architectures.

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from market import get_share_price
from polygon import RESTClient
import os

load_dotenv(override=True)

## Phase 1: Local Memory Server (Knowledge Graph)

Persistent entity-relationship memory using LibSQL.

In [ ]:
# Initialize Memory MCP Server
params = {
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": {"LIBSQL_URL": "file:./memory/user.db"}
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
# Store Information in Memory
instructions = "Use entity tools to store and recall information from conversations."
request = """I'm an LLM engineer building AI agent systems. 
I'm particularly interested in MCP protocol for tool integration."""
model = "gpt-4o-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="memory_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("store_memory"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
# Recall from Memory
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
   agent = Agent(name="memory_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("recall_memory"):
        result = await Runner.run(agent, "What do you know about me?")
    display(Markdown(result.final_output))

## Phase 2: Local + API Integration (Brave Search)

In [ ]:
# Configure Brave Search MCP Server
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {
    "command": "npx", 
    "args": ["-y", "@modelcontextprotocol/server-brave-search"], 
    "env": env
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
# Execute Web Research
instructions = "Search the web for information and summarize findings."
request = f"""Research the latest news on Amazon stock price. 
Current date: {datetime.now().strftime('%Y-%m-%d')}"""

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="researcher", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("web_research"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Phase 3: Financial Data Integration (Polygon.io)

**Setup:** Sign up at [polygon.io](https://polygon.io), obtain API key, and add to `.env`:
```
POLYGON_API_KEY=your_key_here
POLYGON_PLAN=free  # or 'paid' or 'realtime'
```

In [ ]:
# Verify Polygon API Configuration
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("⚠️ POLYGON_API_KEY not set in .env")
else:
    print("✓ Polygon API configured")

In [ ]:
# Test Direct Polygon API
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

In [ ]:
# Custom Market Module (Rate Limit Optimization)
# Caches EOD prices to avoid API rate limits
get_share_price("AAPL")

### Custom Market MCP Server

In [ ]:
# Use Custom MCP Server
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
# Agent with Market Data
instructions = "Answer questions about stock market prices."
request = "What's the current share price of Apple?"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="market_analyst", instructions=instructions, model="gpt-4o-mini", mcp_servers=[mcp_server])
    with trace("market_query"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### Polygon.io Official MCP Server (Paid Plans)

For advanced features with paid subscriptions:

In [ ]:
# Official Polygon MCP Server
params = {
    "command": "uvx",
    "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
    "env": {"POLYGON_API_KEY": polygon_api_key}
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

print(f"Available tools: {len(mcp_tools)}")
mcp_tools[:5]  # Show first 5

In [ ]:
# Use Specific Tool
instructions = "Answer stock market questions using snapshot data."
request = "What's Apple's current stock price? Use get_snapshot_ticker."

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="market_analyst", instructions=instructions, model="gpt-4o-mini", mcp_servers=[mcp_server])
    with trace("polygon_query"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Configuration Management

Update `.env` to specify Polygon plan:
```
POLYGON_PLAN=paid      # For 15-min delayed data
POLYGON_PLAN=realtime  # For real-time streaming
```

In [ ]:
# Check Configuration
load_dotenv(override=True)
polygon_plan = os.getenv("POLYGON_PLAN")

if polygon_plan == "paid":
    print("📊 Using Polygon paid plan (15-min delay)")
elif polygon_plan == "realtime":
    print("⚡ Using Polygon realtime plan")
else:
    print("📅 Using Polygon free plan (EOD data)")